Cell 1 — Environment bootstrap (must run first)

In [1]:
# ===== MUST RUN FIRST CELL (before importing pyterrier / jnius) =====
import os, glob

# A. Prefer system JDK (if it really exists)
sys_java_home = "/usr/lib/jvm/java-17-openjdk-amd64"
cand = [
    os.path.join(sys_java_home, "lib", "server", "libjvm.so"),
    "/gz-data/jdk/current/lib/server/libjvm.so",  # If you installed to /gz-data/jdk/current as per previous steps
]

jvm_path = next((p for p in cand if os.path.isfile(p)), None)

if jvm_path is None:
    # Fallback: Search in common directories
    for base in ("/usr/lib/jvm", "/gz-data/jdk"):
        for p in glob.glob(base + "/**/lib/server/libjvm.so", recursive=True):
            jvm_path = p
            break

if not jvm_path:
    raise RuntimeError("Cannot find libjvm.so. Please install JDK 17 (system or /gz-data) and try again.")

java_home = os.path.abspath(os.path.join(jvm_path, "..", ".."))  # Remove /lib/server
os.environ["JAVA_HOME"] = java_home
os.environ["JVM_PATH"]  = jvm_path
os.environ["LD_LIBRARY_PATH"] = os.path.dirname(jvm_path) + ":" + os.environ.get("LD_LIBRARY_PATH", "")
os.environ["PATH"] = os.path.join(java_home, "bin") + ":" + os.environ.get("PATH", "")

# Optional: Specify Java temp directory and memory
os.environ.setdefault("JAVA_TOOL_OPTIONS", "-Djava.io.tmpdir=/gz-data/tmp")
os.environ.setdefault("_JAVA_OPTIONS", "-Xms512m -Xmx8g")
os.makedirs("/gz-data/tmp", exist_ok=True)

print("JAVA_HOME =", os.environ["JAVA_HOME"])
print("JVM_PATH  =", os.environ["JVM_PATH"])

# ===== Now import PyTerrier and initialize JVM =====
import pyterrier as pt
if not pt.started():
    # Use traditional init for stability (includes Java initialization); add mem / jvm_opts if needed
    pt.init()  # Or pt.init(tail=False)
print("PyTerrier started =", pt.started())

JAVA_HOME = /gz-data/jdk/current/lib
JVM_PATH  = /gz-data/jdk/current/lib/server/libjvm.so


/tmp/ipykernel_670/1771033048.py:39: DeprecationWarning: Call to deprecated function (or staticmethod) started. (use pt.java.started() instead) -- Deprecated since version 0.11.0.
  if not pt.started():
Picked up JAVA_TOOL_OPTIONS: -Djava.io.tmpdir=/gz-data/tmp
Picked up _JAVA_OPTIONS: -Xms512m -Xmx8g


PyTerrier started = True


Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]
/tmp/ipykernel_670/1771033048.py:41: DeprecationWarning: Call to deprecated method pt.init(). Deprecated since version 0.11.0.
java is now started automatically with default settings. To force initialisation early, run:
pt.java.init() # optional, forces java initialisation
  pt.init()  # 或 pt.init(tail=False)
/tmp/ipykernel_670/1771033048.py:42: DeprecationWarning: Call to deprecated function (or staticmethod) started. (use pt.java.started() instead) -- Deprecated since version 0.11.0.
  print("PyTerrier started =", pt.started())


Cell 3 — Stage-1 (context-free)

In [5]:
# ========== Environment & Pipeline Setup ==========
import os, re, torch, random, warnings, pandas as pd
from datasets import load_dataset
import pyterrier as pt
from colbert import Indexer
from colbert.infra import Run, RunConfig, ColBERTConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from tqdm import tqdm
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
from bert_score import score as bert_score
from colbert.modeling.checkpoint import Checkpoint
from colbert.modeling.colbert import colbert_score

warnings.filterwarnings("ignore")

# ====== Cache & Path Config ======
os.environ["HF_HOME"] = "/gz-data/hf_cache"
os.environ["TRANSFORMERS_CACHE"] = "/gz-data/hf_cache"
os.environ["HF_DATASETS_CACHE"] = "/gz-data/hf_cache"
os.makedirs("/gz-data/hf_cache", exist_ok=True)
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

ROOT = "/gz-data/nlquad_colbert"
os.makedirs(ROOT, exist_ok=True)
BM25_TOPK = 5
MAXLEN, GEN_MAXLEN = 250, 384
MIN_SPLIT_LEN, DESIRED_SEG_LEN = 1000, 250
COLBERT_MODEL = "/gz-data/models/colbertv2.0"
GEN_MODEL = "/gz-data/models/deepseek-llm-7b-chat"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

random.seed(42); torch.manual_seed(42)
if not pt.started():
    pt.init()

# ===== Load Data & Preprocess =====
print(">>> Loading NLQuAD...")
dataset = load_dataset("LLukas22/NLQuAD", split="test")
records = []
for art in dataset:
    for para in art["paragraphs"]:
        ctx = para["context"]
        if len(ctx.split()) >= MIN_SPLIT_LEN:
            cid = para["qas"][0]["id"].split("_")[0]
            for qa in para["qas"]:
                if qa["answers"]:
                    records.append({
                        "context_id": cid,
                        "context": ctx,
                        "question": qa["question"],
                        "answer": qa["answers"][0]["text"],
                        "qa_id": qa["id"]
                    })
df = pd.DataFrame(records)
df = df.sort_values(["context_id", "qa_id"]).reset_index(drop=True)

# ===== Paragraph Splitting =====
def semantic_split(text, max_words=DESIRED_SEG_LEN):
    sents = re.split(r"(?<=[.!?])\s+", text.strip())
    buf, out = [], []
    for s in sents:
        if s.strip():
            buf.append(s)
            if len(" ".join(buf).split()) >= max_words:
                out.append(" ".join(buf))
                buf = []
    if buf: out.append(" ".join(buf))
    return out

para_records = []
for cid, grp in df.groupby("context_id"):
    context = grp["context"].iloc[0]
    for i, seg in enumerate(semantic_split(context)):
        para_records.append({"docno": f"{cid}_{i}", "text": seg, "cid": cid})
para_df = pd.DataFrame(para_records)
para_df["docid"] = para_df.index.astype(str)
docno_to_docid = dict(zip(para_df["docno"], para_df["docid"]))
para_text_map = dict(zip(para_df["docid"], para_df["text"]))

def clean_query(q):
    return re.sub(r"[^A-Za-z0-9 ]", "", q.strip())

# ===== Read eligible qids (S5 unified set) =====
ELIGIBLE_CSV = f"{ROOT}/eligible_qids_top5.csv"
eligible = None
if os.path.exists(ELIGIBLE_CSV):
    try:
        eligible = set(pd.read_csv(ELIGIBLE_CSV)["qa_id"].astype(str))
        print(f">>> Loaded eligible S5 set: {len(eligible)} qids")
    except Exception as e:
        print(f"⚠️ Failed to load {ELIGIBLE_CSV}: {e}. Will fallback to >=5 check.")

# ===== ColBERT / BM25 (not used for baseline; kept commented to preserve structure) =====
# print(">>> Building ColBERT Index...")
# with Run().context(RunConfig(nranks=1, root=ROOT, experiment="nlquad", index_root=ROOT)):
#     conf = ColBERTConfig(nbits=2, root=ROOT, doc_maxlen=MAXLEN)
#     idxer = Indexer(checkpoint=COLBERT_MODEL, config=conf)
#     idxer.index(name="nlquad", collection=para_df["text"].tolist(), overwrite="reuse")
#     checkpoint = Checkpoint(COLBERT_MODEL, colbert_config=conf)

# print(">>> Building BM25 Index...")
# index_ref = f"{ROOT}/pt_index"
# if not os.path.exists(index_ref):
#     index_ref = pt.IterDictIndexer(f"{ROOT}/pt_index", meta={"docno":44,"text":60000}, overwrite=True).index(para_df.to_dict("records"))
# index = pt.IndexFactory.of(index_ref)
# bm25 = pt.BatchRetrieve(index, wmodel="BM25")

# ===== Load LLM =====
print(">>> Loading LLM...")
tokenizer = AutoTokenizer.from_pretrained(GEN_MODEL, trust_remote_code=True)
quant_cfg = BitsAndBytesConfig(load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained(
    GEN_MODEL,
    quantization_config=quant_cfg,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)
torch.backends.cuda.matmul.allow_tf32 = True
model = torch.compile(model, mode="reduce-overhead")
print(">>> Model ready!")

# ===== Prompt Template (baseline: no context) =====
def build_prompt_baseline(question, context):
    return f"""You are an AI assistant. Based on the context, answer the question in the following format:

Context: None

Question: {question}

Final Answer:"""

# ===== Dynamic Batch Inference =====
def batch_generate_dynamic(prompts, initial_bs=16, max_bs=64):
    results = []
    i, bs = 0, initial_bs
    last_safe_bs = initial_bs
    while i < len(prompts):
        batch = prompts[i:i+bs]
        try:
            inputs = tokenizer(batch, return_tensors="pt", padding=True,
                               truncation=True, max_length=2048).to(DEVICE)
            outputs = model.generate(
                **inputs,
                max_new_tokens=GEN_MAXLEN,
                min_new_tokens=256,
                length_penalty=1.2,
                pad_token_id=tokenizer.eos_token_id
            )
            decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
            results.extend([d.strip() for d in decoded])
            i += bs
            if bs < max_bs:
                last_safe_bs = bs
                bs = min(bs * 2, max_bs)
        except RuntimeError as e:
            if "CUDA out of memory" in str(e):
                torch.cuda.empty_cache()
                print(f"⚠️ OOM at bs={bs}, rollback to {last_safe_bs}")
                bs = max(last_safe_bs // 2, 1)
                if bs < 1:
                    print("❌ Even bs=1 failed, aborting.")
                    break
            else:
                raise
    return results

# ===== Metric Calculation =====
def compute_metrics(gens, refs):
    scorer = rouge_scorer.RougeScorer(["rouge1","rouge2","rougeL"], use_stemmer=True)
    _, _, bert_f1 = bert_score(gens, refs, lang="en", model_type="roberta-large", verbose=False)
    res = []
    for g, r, b in zip(gens, refs, bert_f1):
        scr = scorer.score(r, g)
        bleu = sentence_bleu([r.split()], g.split())
        res.append({
            "rouge1": round(scr["rouge1"].fmeasure, 4),
            "rouge2": round(scr["rouge2"].fmeasure, 4),
            "rougeL": round(scr["rougeL"].fmeasure, 4),
            "bleu": round(bleu, 4),
            "bertscore": round(b.item(), 4)
        })
    return res

# ===== Answer Cleaning =====
def clean_answer(text):
    text = text.strip()
    if "Final Answer:" in text:
        return text.split("Final Answer:", 1)[1].strip()
    paragraphs = re.split(r"\n\s*\n", text)
    cleaned_paras = []
    for para in paragraphs:
        first_line = para.strip().splitlines()[0] if para.strip() else ""
        if first_line.startswith(("Question:", "Context:", "RULES:", "Answer the question")):
            continue
        cleaned_paras.append(para.strip())
    return "\n\n".join(p for p in cleaned_paras if p)

# ===== Top-K Configuration =====
CTX_TOPK_LIST = [1, 2, 3, 4, 5]
topk_list = [(k, f"top{k}") for k in CTX_TOPK_LIST]
MAX_REQUIRED_K = max(CTX_TOPK_LIST)  # 5

# ===== Main Loop (baseline: one generation per question, no context) =====
results = []

for cid, grp in tqdm(df.groupby("context_id")):
    batch_prompts, meta = [], []
    for _, row in grp.iterrows():
        q, gt, qid = row["question"], row["answer"], str(row["qa_id"])

        if eligible is not None and qid not in eligible:
            continue

        # No-context baseline
        batch_prompts.append(build_prompt_baseline(q))
        meta.append(("baseline", gt, row["qa_id"], q, 0, "", cid))

    if not batch_prompts:
        continue

    answers = batch_generate_dynamic(batch_prompts, initial_bs=16, max_bs=64)
    for ans, (strat, gt, qid, q, topk_val, topk_ranked_ctx, cid_val) in zip(answers, meta):
        if not ans:
            continue
        ans_clean = clean_answer(ans)
        m = compute_metrics([ans_clean], [gt])[0]
        results.append({
            "cid": cid_val,
            "qid": qid,
            "question": q,
            "topk": topk_val,                         # 0
            "topk_ranked_context": topk_ranked_ctx,   # empty
            "strategy": strat,                        # "baseline"
            "answer_clean": ans.strip(),
            "answer_for_eval": ans_clean,
            **m
        })

# ===== Save Results =====
df_res = pd.DataFrame(results)

# Fixed column order
cols_order = [
    "cid","qid","question",
    "topk","topk_ranked_context","strategy",
    "answer_clean","answer_for_eval",
    "rouge1","rouge2","rougeL","bleu","bertscore"
]
df_res = df_res[cols_order]

# Convert \n in topk_ranked_context to CRLF so it auto-wraps in Excel/tables
df_res["topk_ranked_context"] = df_res["topk_ranked_context"].apply(lambda x: x.replace("\n", "\r\n"))

df_res.to_csv(f"{ROOT}/final_results_stage1.csv", index=False)
avg_m = df_res.groupby("strategy")[["rouge1","rouge2","rougeL","bleu","bertscore"]].mean().reset_index()
avg_m.to_csv(f"{ROOT}/average_metrics_stage1.csv", index=False)

print("✅ Done. Baseline (no context) written to final_results_stage1.csv / average_metrics_stage1.csv")


>>> Loading NLQuAD...
>>> Building ColBERT Index...


[Aug 13, 17:26:36] #> Note: Output directory /gz-data/nlquad_colbert/nlquad already exists


>>> Building BM25 Index...
>>> Loading LLM...


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

>>> Model ready!


  0%|          | 1/235 [00:00<00:39,  5.93it/s]

...

⚠️ OOM at bs=16, rollback to 16


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['ro

⚠️ OOM at bs=16, rollback to 16
⚠️ OOM at bs=16, rollback to 8


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['ro

✅ Done. Added reversed for top-2 and `topk_ranked_context` column (rank1/2/... by ColBERT).


Cell 4 — Stage-2 (LLM judging): coherence & informativeness → per-sample scores

In [7]:
# ========== Stage 2: LLM-based Scoring (Coherence / Informativeness) ==========
import os, re, gc, warnings
import torch
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM

warnings.filterwarnings("ignore")

# ====== Paths ======
ROOT = "/gz-data/nlquad_colbert"
STAGE1_FILE = f"{ROOT}/final_results_stage1-baseline.csv"     # input from Stage-1 (baseline or BM25/ColBERT)
STAGE2_FILE = f"{ROOT}/final_results_stage2-baseline.csv"     # output path
LLM_SCORE_MODEL = "/gz-data/models/qwen2.5"          # change to your local scorer (e.g., Qwen2.5-7B-Instruct)

# ====== GPU memory guard (optional but helpful) ======
def auto_max_memory():
    """
    Returns a dict like {0: 'XXGiB', 1: 'XXGiB'} to cap per-GPU memory and reduce OOM risk.
    Uses (total - 1GiB) per GPU.
    """
    if not torch.cuda.is_available():
        return "auto"
    mem = {}
    for i in range(torch.cuda.device_count()):
        total_gb = int(torch.cuda.get_device_properties(i).total_memory / (1024**3))
        mem[i] = f"{max(total_gb - 1, 1)}GiB"
    return mem

# ====== Build gold answers map from NLQuAD ======
print(">>> Loading NLQuAD (test) to recover gold answers ...")
dataset = load_dataset("LLukas22/NLQuAD", split="test")
gold_map = {}
for article in dataset:
    for para in article["paragraphs"]:
        for qa in para["qas"]:
            if qa.get("answers"):
                gold_map[str(qa["id"])] = qa["answers"][0]["text"]

# ====== Load Stage-1 results ======
print(f">>> Reading Stage-1 file: {STAGE1_FILE}")
df = pd.read_csv(STAGE1_FILE)
if "qid" not in df.columns:
    raise ValueError("Stage-1 file must include a 'qid' column.")
if "answer_for_eval" not in df.columns and "answer_clean" in df.columns:
    # Backward-compat fallback
    df["answer_for_eval"] = df["answer_clean"]

# Inject gold text
df["gold_answer"] = df["qid"].astype(str).map(gold_map).fillna("")

# ====== Load scorer (Qwen) ======
print(">>> Loading scorer model ...")
tok = AutoTokenizer.from_pretrained(LLM_SCORE_MODEL, trust_remote_code=True)
score_model = AutoModelForCausalLM.from_pretrained(
    LLM_SCORE_MODEL,
    torch_dtype=torch.float16,
    device_map="balanced" if torch.cuda.is_available() else "auto",
    max_memory=auto_max_memory()
)
print(">>> Scorer ready!")

# ====== Prompt ======
def build_score_prompt(question: str, gold: str, generated: str) -> str:
    return f"""
You are an expert evaluator. Based on the question, gold answer, and generated answer, provide two scores:
1. Coherence (1-5): Logical clarity and fluency of the generated answer.
2. Informativeness (1-5): Amount of correct and relevant information compared to the gold answer.

Question: {question}
Gold Answer: {gold}
Generated Answer: {generated}

IMPORTANT:
- Do NOT provide explanations or extra text.
- Output ONLY two integers in the LAST line, separated by a comma.
Format:
x,y

Now output the result:
""".strip()

# ====== Output parser ======
def parse_llm_output(text: str):
    # Prefer a clean "x,y" on the last line
    lines = (text or "").strip().splitlines()
    for line in reversed(lines):
        if re.match(r"^\s*[1-5]\s*,\s*[1-5]\s*$", line):
            a, b = [int(x.strip()) for x in line.split(",")]
            return {"coherence": a, "informativeness": b}
    # Fallback: find last two numbers 1..5
    nums = re.findall(r"\b[1-5]\b", text)
    if len(nums) >= 2:
        return {"coherence": int(nums[-2]), "informativeness": int(nums[-1])}
    return {"coherence": 0, "informativeness": 0}

# ====== Batched generation with safe fallback ======
def generate_scores_batched(prompts, initial_bs=8, min_bs=1, max_new_tokens=64):
    results = []
    i, bs = 0, initial_bs
    while i < len(prompts):
        batch = prompts[i:i+bs]
        try:
            inputs = tok(batch, return_tensors="pt", padding=True, truncation=True).to(score_model.device)
            with torch.inference_mode():
                outputs = score_model.generate(
                    **inputs,
                    max_new_tokens=max_new_tokens,
                    do_sample=False,
                    pad_token_id=tok.eos_token_id
                )
            decoded = tok.batch_decode(outputs, skip_special_tokens=True)
            results.extend(decoded)
            i += bs
            # Optional: try to climb back if we had reduced bs earlier
            if bs < initial_bs:
                bs = min(bs * 2, initial_bs)
        except RuntimeError as e:
            if "CUDA out of memory" in str(e):
                torch.cuda.empty_cache()
                new_bs = max(bs // 2, min_bs)
                print(f"⚠️ OOM at batch_size={bs}; reducing to {new_bs}")
                if new_bs == bs == 1:
                    raise RuntimeError("Even batch_size=1 failed due to OOM.")
                bs = new_bs
            else:
                raise
    return results

# ====== Stage-2 main ======
print(">>> Stage 2 (LLM-based scoring) running ...")
BATCH = 8
prompts, metas, rows = [], [], []

for r in tqdm(df.itertuples(), total=len(df)):
    question = getattr(r, "question", "")
    gold     = getattr(r, "gold_answer", "")
    pred     = getattr(r, "answer_for_eval", getattr(r, "answer_clean", ""))

    prompts.append(build_score_prompt(str(question), str(gold), str(pred)))
    metas.append(r._asdict())

    if len(prompts) >= BATCH:
        outs = generate_scores_batched(prompts, initial_bs=BATCH)
        for out, meta in zip(outs, metas):
            sc = parse_llm_output(out)
            meta.update(sc)
            rows.append(meta)
        prompts, metas = [], []

# Flush remaining
if prompts:
    outs = generate_scores_batched(prompts, initial_bs=BATCH)
    for out, meta in zip(outs, metas):
        sc = parse_llm_output(out)
        meta.update(sc)
        rows.append(meta)

# ====== Save ======
df_s2 = pd.DataFrame(rows)
df_s2.to_csv(STAGE2_FILE, index=False)
print(f"✅ Stage 2 completed. Saved: {STAGE2_FILE}")


>>> Loading NLQuAD to recover gold answers ...


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


>>> Loading scorer (Qwen2.5-7B-Instruct) on multi-GPU ...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

>>> Scorer ready!
>>> Stage 2 (LLM-based scoring) running ...


...

⚠️ OOM at batch_size=8, reducing to 4


 32%|███▏      | 904/2796 [05:33<15:29,  2.04it/s]

⚠️ OOM at batch_size=8, reducing to 4


100%|██████████| 2796/2796 [17:10<00:00,  2.71it/s]


✅ Stage 2 completed. Results saved at /gz-data/nlquad_colbert/final_results_stage2.csv


Cell 5 — Merge baseline vs Top-K (strictly paired on qid)

In [9]:
import pandas as pd
import re
import os

ROOT = "/gz-data/nlquad_colbert"
BASELINE_STAGE2 = f"{ROOT}/final_results_stage2-baseline.csv"
TOPK_STAGE2     = f"{ROOT}/final_results_stage2.csv"
MERGED_FILE     = f"{ROOT}/final_results_stage2_merged.csv"

# 1) Read
b = pd.read_csv(BASELINE_STAGE2, dtype={"qid": str, "cid": str})
t = pd.read_csv(TOPK_STAGE2,     dtype={"qid": str, "cid": str})

# 2) Keep only qids from baseline that also appear in topK
keep_qids = set(t["qid"].astype(str))
b = b[b["qid"].astype(str).isin(keep_qids)].copy()

# 3) Column union & alignment
all_cols = list(dict.fromkeys(list(b.columns) + list(t.columns)))  # Preserve original order
for col in all_cols:
    if col not in b.columns:
        b[col] = pd.NA
    if col not in t.columns:
        t[col] = pd.NA

# 4) Fill missing fields
if "strategy" not in b.columns:
    b["strategy"] = "baseline"
if "topk" not in b.columns:
    b["topk"] = pd.NA
if "topk" not in t.columns:
    t["topk"] = t["strategy"].str.extract(r'top(\d+)').astype(float)

# 5) Merge
df = pd.concat([b[all_cols], t[all_cols]], ignore_index=True)

# 6) Sort
def parse_topk_from_any(row):
    k = row.get("topk", pd.NA)
    if pd.isna(k):
        m = re.search(r"top(\d+)", str(row.get("strategy", "")))
        return int(m.group(1)) if m else None
    try:
        return int(k)
    except Exception:
        return None

def strat_rank(row):
    s = str(row.get("strategy", ""))
    if s == "baseline":
        return (0, 0)

    k = parse_topk_from_any(row)
    if k is None:
        return (999, 0)

    base = 1 + (k - 1) * 3
    if "sequential" in s:
        off = 0
    elif "reversed" in s:
        off = 1
    elif "shuffled" in s:
        off = 2
    else:
        off = 3
    return (base + off, 0)

df["qid"] = df["qid"].astype(str)
df["__rank"] = df.apply(strat_rank, axis=1)
df = df.sort_values(by=["qid", "__rank"]).drop(columns="__rank")

# 7) Save
df.to_csv(MERGED_FILE, index=False, encoding="utf-8")
print(f"✅ Merge completed, saved to: {MERGED_FILE}")
print(df.head(10))

✅ 合并完成，保存到: /gz-data/nlquad_colbert/final_results_stage2_merged.csv
      cid     qid                                 question         strategy  \
0    1018  1018_0  Where are Syrian migrants trying to go?         baseline   
233  1018  1018_0  Where are Syrian migrants trying to go?  sequential_top1   
234  1018  1018_0  Where are Syrian migrants trying to go?  sequential_top2   
235  1018  1018_0  Where are Syrian migrants trying to go?    reversed_top2   
236  1018  1018_0  Where are Syrian migrants trying to go?  sequential_top3   
237  1018  1018_0  Where are Syrian migrants trying to go?    reversed_top3   
238  1018  1018_0  Where are Syrian migrants trying to go?    shuffled_top3   
239  1018  1018_0  Where are Syrian migrants trying to go?  sequential_top4   
240  1018  1018_0  Where are Syrian migrants trying to go?    reversed_top4   
241  1018  1018_0  Where are Syrian migrants trying to go?    shuffled_top4   

                                          answer_clean  \
0   